In [1]:
import numpy as np
import pandas as pd

In [2]:
def getSortedByProbability(original_shape, candidate_mat, probability_mat):
    sorted_candidate_mat = np.zeros((original_shape)).astype('int')
    sorted_probability_mat = np.zeros((original_shape))
    for i in range(original_shape[0]):
        sorted_index = probability_mat[i].argsort()[::-1]
        sorted_candidate_mat[i] = candidate_mat[i, sorted_index]
        sorted_probability_mat[i] = probability_mat[i, sorted_index] 
    return (sorted_candidate_mat, sorted_probability_mat)

In [6]:
%%time
k = 20
split_pair_path = "../data/sep_9_lemonbear_triplet_1epoch/pair_test_k_20_siamese_bert_base_cased_triplet_epoch1.txt"

split_data = pd.read_csv(split_pair_path, sep='\t')

CPU times: user 1.03 s, sys: 47.7 ms, total: 1.08 s
Wall time: 783 ms


In [7]:
%%time
total_len = len(split_data)
unique_len = int(total_len/k)
original_shape = (unique_len, k)
true_sentence = np.array(split_data.true_sentence).reshape(original_shape)
candidate_sentence = np.array(split_data.candidate_sentence).reshape(original_shape)
candidate_label = np.array(split_data.candidate_label).reshape(original_shape)

CPU times: user 268 ms, sys: 8.01 ms, total: 276 ms
Wall time: 60.5 ms


In [8]:
%%time
# from probability file, get top k candidates
probability_file_path = "../data/sep_9_lemonbear_triplet_1epoch/begin_again_2nd_iter_k_50_623w_1epoch_test_results.tsv"

probability_file = pd.read_csv(probability_file_path, sep='\t',header=None).iloc[:,1]
probability_mat = np.array(probability_file).reshape(original_shape)

CPU times: user 125 ms, sys: 3.9 ms, total: 129 ms
Wall time: 109 ms


In [9]:
%%time
# Sort according to probability, descending
sorted_candidate_label, sorted_probability_mat = \
        getSortedByProbability(original_shape,
                                candidate_label,
                                probability_mat)

CPU times: user 157 ms, sys: 25 µs, total: 157 ms
Wall time: 155 ms


In [114]:
line_num = 0
arg_index = np.argmax(probability_mat[line_num,:])
print("True: {0}".format(true_sentence[line_num,0]))
print("Candidate: {0}".format(candidate_sentence[line_num, arg_index]))
print("Candidate Label: {0}".format(candidate_label[line_num, arg_index]))
print("Probability: {0}".format(probability_mat[line_num, arg_index]))

True: Some people say that rappers don’t have feelings. We have feelings. Some people say that we are not rappers. That hurts our feelings.
Candidate: In the words of Reggie Noble aka Redman : "I'll Be Dat"
Candidate Label: 5032
Probability: 0.12616585


0.9995931999999998

In [10]:
output_path = "../data/sep_9_lemonbear_triplet_1epoch/submit_result_lemonbear_triplet_1epoch_k_20_test"
np.save(output_path, sorted_candidate_label[:,0])

In [2]:
k = 100
split_pair_path = "../data/split_test/new_split_pair_test_k_{0}.txt".format(k)
# train_data_path = "../data/split_test/train_set.txt"
test_data_path = "../data/split_test/test_tweets_unlabeled_dataframe_v1_4.txt"


# train_data = pd.read_csv(train_data_path, sep='\t', header=None)
# train_label = np.array(train_data.iloc[:,0]).astype('int')
test_data = pd.read_csv(test_data_path)
split_data = pd.read_csv(split_pair_path, sep='\t')

In [3]:
# from probability file, get top k candidates
probability_file_path = "../data/split_test/test_results_k_100.csv"

probability_file = pd.read_csv(probability_file_path, sep='\t',header=None).iloc[:,1]


In [4]:
%%time
top = k
new_top = top
probability_threshold = 1
new_candidate_list = [None] * int(len(split_data)/top)
counter = 0
for i in range(0, len(split_data), top):
    new_train_data_single_split = split_data[i:(i+top)]
    new_train_label_single_split = np.array(new_train_data_single_split.candidate_label)
    probability_single_split = probability_file[i:(i+top)]
    sorted_index = np.array(np.argsort(probability_single_split)[::-1][:new_top])
    sorted_probability_split = np.array(probability_single_split)[sorted_index]
    new_train_label_single_split = new_train_label_single_split[sorted_index]
    filted_probability_split = sorted_probability_split[sorted_probability_split > probability_threshold]
    candidate_label = np.array([new_train_label_single_split[0]])
    if 0 != len(filted_probability_split):
        candidate_label = new_train_label_single_split[:len(filted_probability_split)]
        #new_candidate_list.append(candidate_label)
    new_candidate_list[counter] = candidate_label
    counter += 1
    

CPU times: user 18 s, sys: 12.8 ms, total: 18 s
Wall time: 18 s


In [5]:
new_top_predict_label_list = np.zeros((len(new_candidate_list), 1)).astype('int')
for i in range(0, len(new_candidate_list)):
    predict_label = np.argmax(np.bincount(new_candidate_list[i]))
    new_top_predict_label_list[i] = predict_label

In [6]:
output_path = "../data/split_test/test_predict_label_k_{0}".format(k)
np.save(output_path, new_top_predict_label_list)

In [7]:
new_top_predict_label_list

array([[6353],
       [ 249],
       [6492],
       ...,
       [4181],
       [9205],
       [  82]])